<a href="https://colab.research.google.com/github/fikrifaizz/llm-task-instruction-generator/blob/main/src/fine_tuning/finetune_sft_peft.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q torch transformers datasets accelerate peft trl

In [ ]:
import torch
import json
from datasets import Dataset
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments # Added TrainingArguments
from trl import SFTConfig, SFTTrainer
from peft import LoraConfig
from trl import setup_chat_format

In [3]:
device = "mps" if torch.backends.mps.is_available() else "cuda" if torch.cuda.is_available() else "cpu"
print(f"Device: {device}")

Device: cpu


In [18]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
model_name = "microsoft/Phi-3.5-mini-instruct"
print(f"Loading {model_name}...")

tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float32,      # Use float32 for CPU (better performance)
    trust_remote_code=True,
    low_cpu_mem_usage=True,         # Essential for CPU
    use_cache=False                 # Disable cache to save memory
)

# Move to CPU explicitly
model = model.to(device)

# Fix tokenizer
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

Loading microsoft/Phi-3.5-mini-instruct...


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
print("Loading dataset...")

with open("dataset.json", "r", encoding="utf-8") as f:
    data = json.load(f)

# Convert to chat format
formatted_data = []
for item in data:
    messages = [
        {
            "role": "system",
            "content": "You are TaskMaster, an expert at creating step-by-step instructions for digital tasks."
        },
        {
            "role": "user",
            "content": f"Create step-by-step instructions for: {item['user_intent']}"
        },
        {
            "role": "assistant",
            "content": "\n".join(item['structured_instructions'])
        }
    ]
    formatted_data.append({"messages": messages})

# Create dataset
dataset = Dataset.from_list(formatted_data)
dataset = dataset.train_test_split(test_size=0.1, seed=42)
print(f"Dataset loaded: {len(dataset['train'])} train, {len(dataset['test'])} test")

In [ ]:
peft_config = LoraConfig(
    r=8,                    # Very small rank for CPU
    lora_alpha=16,         # Proportional alpha
    lora_dropout=0.1,
    target_modules=["q_proj"],  # Only one module type to reduce computation
    task_type="CAUSAL_LM",
    bias="none"
)

In [22]:
from transformers import DataCollatorForLanguageModeling
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False  # We're doing causal LM, not masked LM
)

In [23]:
batch_size = 8
training_args = SFTConfig(
    # Output settings
    output_dir="./phi35-instructions",  # Directory to save model checkpoints
    # Training duration
    num_train_epochs=1,  # Number of training epochs
    # Batch size settings
    per_device_train_batch_size=2,  # Batch size per GPU
    gradient_accumulation_steps=2,  # Accumulate gradients for larger effective batch
    # Memory optimization
    gradient_checkpointing=True,  # Trade compute for memory savings
    # Optimizer settings
    optim="adamw_torch_fused",  # Use fused AdamW for efficiency
    learning_rate=2e-4,  # Learning rate (QLoRA paper)
    max_grad_norm=0.3,  # Gradient clipping threshold
    # Learning rate schedule
    warmup_ratio=0.03,  # Portion of steps for warmup
    lr_scheduler_type="constant",  # Keep learning rate constant after warmup
    # Logging and saving
    logging_steps=10,  # Log metrics every N steps
    save_strategy="epoch",  # Save checkpoint every epoch
    # Precision settings
    bf16=True,  # Use bfloat16 precision
    # Integration settings
    push_to_hub=False,  # Don't push to HuggingFace Hub
    report_to="none",  # Disable external logging
)

In [25]:
trainer = SFTTrainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    peft_config=peft_config,
    data_collator = data_collator
)

print("Trainer created")

/usr/local/lib/python3.11/dist-packages/peft/mapping_func.py:73: UserWarning: You are trying to modify a model with PEFT for a second time. If you want to reload the model with a different config, make sure to call `.unload()` before.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/peft/tuners/tuners_utils.py:190: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(


Tokenizing train dataset:   0%|          | 0/433 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/433 [00:00<?, ? examples/s]

Tokenizing eval dataset:   0%|          | 0/49 [00:00<?, ? examples/s]

Truncating eval dataset:   0%|          | 0/49 [00:00<?, ? examples/s]

Trainer created


In [26]:
trainer.train()

Step,Training Loss


OutOfMemoryError: CUDA out of memory. Tried to allocate 34.00 MiB. GPU 0 has a total capacity of 14.74 GiB of which 12.12 MiB is free. Process 6581 has 14.73 GiB memory in use. Of the allocated memory 14.35 GiB is allocated by PyTorch, and 246.64 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)